In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

def prepare_sequences(df):
    sequences = []
    labels = []
    
    unique_ids = df['outlier_id'].unique()
    
    for oid in unique_ids:
        prior_data = df[(df['outlier_id'] == oid) & (df['day_type'] == 'prior day')]['vw'].values
        post_data = df[(df['outlier_id'] == oid) & (df['day_type'] == 'post day')]['vw'].values
        
        if len(prior_data) == 72 and len(post_data) == 72:  # Ensure full sequences
            sequences.append(prior_data)
            labels.append(post_data[:3])  # Only the first 3 post days are needed
            
    return np.array(sequences), np.array(labels)

# Load your dataset
df = pd.read_csv('your_dataset.csv')  # Make sure to load your actual data file

# Normalize each sequence
scaler = MinMaxScaler(feature_range=(0, 1))
df['vw'] = scaler.fit_transform(df[['vw']])

X, y = prepare_sequences(df)

# Reshape for LSTM input
X = X.reshape((X.shape[0], X.shape[1], 1))
y = y.reshape((y.shape[0], 3))  # Predicting only the first 3 days of post day

# Define the LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(72, 1)),  # Input sequences are 72 time steps
    LSTM(50),
    Dense(3)  # Output 3 predictions
])

model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

# Split data into train and test sets
train_size = int(len(X) * 0.67)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64, verbose=2)

# Make predictions
predictions = model.predict(X_test)

# Optionally, scale back the predictions to the original scale
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test)

print("Model training complete and predictions made.")